In [1]:
import pandas as pd

# Initialize an empty DataFrame to collect changes
changes_df = pd.DataFrame(columns=master_df.columns)

# Loop over each month's GHA file
for gha_file in gha_files:
    # Load the GHA data
    gha_month_df = pd.read_csv(gha_file)
    
    # Ensure consistent column names in both DataFrames
    gha_month_df = gha_month_df.rename(columns={'Position Number': 'Position ID', 'Employee ID': 'Employee ID_gha'})
    
    # Merge master with current month's GHA data on 'Position ID' and 'Employee ID'
    merged_df = master_df.merge(gha_month_df[['Position ID', 'Employee ID_gha']], 
                                on='Position ID', 
                                how='left', 
                                indicator=True)
    
    # Find rows where 'Employee ID' from GHA is different or new
    new_or_changed = merged_df[(merged_df['_merge'] == 'right_only') | 
                               (merged_df['Employee ID'] != merged_df['Employee ID_gha'])]
    
    # Append new or changed rows to changes_df
    changes_df = pd.concat([changes_df, new_or_changed], ignore_index=True)

# Drop '_merge' column if present
if '_merge' in changes_df.columns:
    changes_df = changes_df.drop(columns='_merge')

# Append changes_df to master_df
master_df = pd.concat([master_df, changes_df], ignore_index=True)

# Ensure all columns are synchronized between master_df and changes_df
master_df = master_df[changes_df.columns]

# Save or print final result
print(master_df)


  Position ID Employee ID  Jan 24  Feb 24  Mar 24
0      000123        E001       1       1       0
1      000456        E002       1       0       0
2      000789        E003       0       0       1
